In [169]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm

In [170]:
dataframe = pd.read_csv("./heart.csv")
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol      1025 non-null   int64  
 5   fbs       1025 non-null   int64  
 6   restecg   1025 non-null   int64  
 7   thalach   1025 non-null   int64  
 8   exang     1025 non-null   int64  
 9   oldpeak   1025 non-null   float64
 10  slope     1025 non-null   int64  
 11  ca        1025 non-null   int64  
 12  thal      1025 non-null   int64  
 13  target    1025 non-null   int64  
dtypes: float64(1), int64(13)
memory usage: 112.2 KB


In [171]:
dataframe["target"] = dataframe["target"].replace(0, -1)
dataframe.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,-1
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,-1
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,-1
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,-1
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,-1


In [172]:
def scalerTranfrom(features):
    new_features = features
    # maximun = new_features.max(axis=0)
    # minimun = new_features.min(axis=0)
    
    # differenct = maximun - minimun
    
    mean = np.mean(new_features, axis=0)
    standard_deviation = np.std(new_features, axis=0)
    
    for i in range(new_features.shape[0]):
        new_features[i] = np.round((new_features[i] - mean) / standard_deviation, 2)
    
    
    return new_features

In [173]:
features = np.array(dataframe.drop(columns="target"))
labels = np.array(dataframe["target"])

# print(features.shape)
# features = scalerTranfrom(features)
# print(features)

features_train, features_test, labels_train, labels_test = train_test_split(features, labels)

In [174]:
import time
# Support Vector Machine classifiers by kernel which is linear.
class SVC:
    def __init__(self, c=1, learning_rate=1):
        self.cost = c
        self.learning_rate = learning_rate
    
    def fit(self, features_train, labels_train):
        
        #store info
        self.d = features_train.shape[1]
        self.n_record = features_train.shape[0]
        
        # set up features and labels
        full_one = np.full((self.n_record, 1), 1)
        self.features = np.append(features_train, full_one, axis=1)
        self.lables = labels_train
        
        # set vector of weight by d+1 dimensions
        self.weight = np.zeros(self.d + 1)
        
        self.weight = self.computeWeight(self.weight, self.features, self.lables, self.n_record) # compute to get weight's value
        
        # array_weight = np.array([])
        # array_weight_cal = np.array([])
        
        # for i in range(10):
        #     # Generating Permutation of feature and label
        #     np.random.seed(i)
        #     new_feature = np.permutation(self.features)
        #     np.random.seed(i)
        #     new_label = np.permutation(self.lables)
            
        #     # calculate weight
        #     new_weight = self.computeSVM(self.weight, new_feature, new_label, self.n_record) # compute to get weight's value
            
        #     # append weight that calculate
        #     array_weight = np.append(array_weight, new_weight, axis=1)
        #     weight_cal = np.inner(new_weight, new_weight)
        #     array_weight_cal = np.append(array_weight_cal, weight_cal)
        
    
    def computeWeight(self, weight, feature_train, labels_train, n_record):
        while True:
            m = 0 # the number of missclassifier
            for i in range(n_record):
                check = labels_train[i] * (np.inner(weight, feature_train[i]))
                if check <= 0:
                    m += 1
                    weight += weight + labels_train[i] * feature_train[i]
            
            if m == 0:
                break
        
        return weight
    
    def predict(self, feature_test):
        check = np.inner(self.weight, feature_test)
        if check > 1:
            # positive class
            return 1 
        else:
            # negative class
            return -1
    
    def hinge_loss(self):
        sum_loss = 0
        for i in range(self.n_record):
            differenct = 1 - self.lables[i] * (np.inner(self.weight, self.features[i]))
            
            # check max[0, differenct]
            if differenct > 0:
                sum_loss += differenct
        return sum_loss
    
    def score(self, features_test, labels_test):
        size = features_test.shape[0]
        full_one = np.full((size, 1), 1)
        new_features = np.append(features_test, full_one, axis=1)
        
        point = 0
        for i in range(size):
            y_predict = self.predict(new_features[i])
            if y_predict == labels_test[i]:
                point += 1
        
        return np.round(point / size, 4)
            

In [175]:
model = SVC()
model.fit(features_train, labels_train)
print(model.weight)

[ 1.01178309e+304 -6.09596887e+304  1.12492012e+305 -1.80176488e+306
 -1.90418145e+306  2.15641107e+302  2.20071830e+304  2.26895582e+306
  4.38890136e+304 -1.45699805e+305 -2.14472648e+302  1.42309946e+300
 -6.43976882e+304 -1.00000000e+000]


In [176]:
model.score(features_test, labels_test)

0.5097

In [177]:
# Model Selection
class ModelSelection:
    def __init__(self, features, labels , model):
        self.features = features
        self.labels = labels
        self.model = model
        pass